In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import sys
sys.path.append('../src')

In [9]:
import os
import torch
import numpy as np
import pandas as pd
import torchvision.transforms as T
from tqdm.notebook import tqdm
from PIL import Image
from pathlib import Path
from torchray.utils import get_device
from IPython.display import clear_output, display, Latex

from models.classifier import VGG16ClassifierModel, Resnet50ClassifierModel

In [10]:
dataset = 'VOC'
num_classes = 20
data_path = Path('../datasets/VOC2007/VOCdevkit/VOC2007/JPEGImages/')
classifier_type = 'vgg16'
classifier_checkpoint = Path('../src/checkpoints/pretrained_classifiers/vgg16_voc.ckpt')

masks_base_path = Path('../src/evaluation/masks/')

In [11]:
methods = ['explainer', 'grad_cam', 'rise', 'rt_saliency']

mask_objects = ['bottle', 'car', 'cat', 'dog', 'person']

In [12]:
target_dict = {'aeroplane' : 0, 'bicycle' : 1, 'bird' : 2, 'boat' : 3, 'bottle' : 4, 'bus' : 5, 'car' : 6, 
                'cat' : 7, 'chair' : 8, 'cow' : 9, 'diningtable' : 10, 'dog' : 11, 'horse' : 12, 'motorbike' : 13, 'person' : 14, 
                'pottedplant' : 15, 'sheep' : 16, 'sofa' : 17, 'train' : 18, 'tvmonitor' : 19}

inv_target_dict = {value: key for key, value in target_dict.items()}

mask_classes = [target_dict[obj] for obj in mask_objects]

In [13]:
class_count_dict = {'aeroplane': 205, 'bicycle': 250, 'bird': 289, 'boat': 176, 'bottle': 240, 'bus': 183, 'car': 775, 
                    'cat': 332, 'chair': 545, 'cow': 127, 'diningtable': 247, 'dog': 433, 'horse': 279, 'motorbike': 233, 
                    'person': 2097, 'pottedplant': 254, 'sheep': 98, 'sofa': 355, 'train': 259, 'tvmonitor': 255}

In [13]:
if classifier_type == 'vgg16':
    model = VGG16ClassifierModel.load_from_checkpoint(classifier_checkpoint, num_classes=num_classes, dataset=dataset)
elif classifier_type == 'resnet50':
    model = Resnet50ClassifierModel.load_from_checkpoint(classifier_checkpoint, num_classes=num_classes, dataset=dataset)
else:
    raise Exception("Unknown classifier type " + classifier_type)
device = get_device()
model.to(device)
model.eval()
for param in model.parameters():
    param.requires_grad_(False)

In [14]:
def compute_results(model, image, mask, class_id):
    thresholds = np.arange(0.1, 1.0, 0.1)
    outputs = []
    for threshold in thresholds:
        thresh_mask = (mask > threshold).float()
        masked_image = thresh_mask * image
        output_probs = torch.nn.Softmax(dim=1)(model(masked_image))
        outputs.append(output_probs[0][class_id].cpu().numpy())

    return np.mean(outputs)

In [14]:
try:
    results = np.load('class_scores_results.npz', allow_pickle=True)['results'].item()
except:
    results = {}

In [24]:
transformer = T.Compose([ T.Resize(size=(224,224)),
                          T.ToTensor(), 
                          T.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])])


for target_class in mask_classes:
    #Only need filenames from the directory, and not masks, therefore we can just take any method here
    masks_dir = (masks_base_path / '{}_{}_{}'.format(dataset, classifier_type, "explainer") 
                                 / 'class_masks'
                                 / 'target_class_{}'.format(target_class)
                                 / 'masks_for_class_{}'.format(target_class))

    clear_output(wait=True)
    print("Evaluating classifier on unmasked {} images".format(inv_target_dict[target_class]))

    all_scores = []
    for filename in tqdm(masks_dir.glob('*.npz'), total=class_count_dict[inv_target_dict[target_class]]):
        jpeg_filename = os.path.splitext(filename.name)[0] + '.jpg'
        image = Image.open(data_path / jpeg_filename).convert("RGB")
        image = transformer(image).unsqueeze(0)
        image = image.to(device)

        output_probs = torch.nn.Softmax(dim=1)(model(image))[0]
        target_prob = output_probs[target_class].cpu().numpy()
        all_scores.append(target_prob)

    for method in methods:
        if method not in results:
            results[method] = {}
        if inv_target_dict[target_class] not in results[method]:
            results[method][inv_target_dict[target_class]] = {}
        
        results[method][inv_target_dict[target_class]]['no_mask'] = {}
        results[method][inv_target_dict[target_class]]['no_mask']['mean_probs'] = np.mean(all_scores)

Evaluating classifier on unmasked person images


In [20]:
transformer = T.Compose([ T.Resize(size=(224,224)),
                          T.ToTensor(), 
                          T.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])])

for method in methods:
    if method not in results:
        results[method] = {}
    for target_class in mask_classes:
        if target_class not in results[method]:
            results[method][inv_target_dict[target_class]] = {}
        for mask_class in mask_classes:
            results[method][inv_target_dict[target_class]][inv_target_dict[mask_class]] = {}

            masks_dir = (masks_base_path / '{}_{}_{}'.format(dataset, classifier_type, method)
                                         / 'class_masks'
                                         / 'target_class_{}'.format(target_class)
                                         / 'masks_for_class_{}'.format(mask_class))

            clear_output(wait=True)
            print("Evaluating method {} for {} images with {} masks".format(method, 
                                                                            inv_target_dict[target_class],
                                                                            inv_target_dict[mask_class]))

            all_scores = []
            for filename in tqdm(masks_dir.glob('*.npz'), total=class_count_dict[inv_target_dict[target_class]]):
                jpeg_filename = os.path.splitext(filename.name)[0] + '.jpg'
                image = Image.open(data_path / jpeg_filename).convert("RGB")
                image = transformer(image).unsqueeze(0)
                image = image.to(device)

                mask = np.load(filename)['arr_0']
                mask = torch.tensor(np.reshape(mask, [1,1, *mask.shape]), device=device)
                score = compute_results(model=model, image=image, mask=mask, class_id=target_class)
                all_scores.append(score)

            results[method][inv_target_dict[target_class]][inv_target_dict[mask_class]]['mean_probs'] = np.mean(all_scores)

Evaluating method rt_saliency for person images with person masks


In [26]:
np.savez('class_scores_results.npz', results=results)

In [40]:
indices = {"no_mask": 0, "bottle": 1, "car": 2, "cat": 3, "dog": 4, "person": 5}

for i, method in enumerate(results):
    num_mask_classes = len(results[method])
    confusion_matrix = np.zeros(shape=(num_mask_classes, num_mask_classes+1))
    for j, target_class in enumerate(results[method]):
        for mask_class in results[method][target_class]:
            index = indices[mask_class]
            confusion_matrix[j, index] = "{:.2f}".format(results[method][target_class][mask_class]['mean_probs'] * 100.0)
    
    print(method)
    df = pd.DataFrame(confusion_matrix, index=mask_objects, columns=["none"] + mask_objects)
    df = df.style.set_caption(method)
    df.format(precision=2)
    df.background_gradient(axis=1, vmin=5, low=0.2, cmap="YlGnBu")
    latex_table = Latex(df.to_latex()).data
    display(df)
    print(latex_table)
    print('\n')
        

explainer


,none,bottle,car,cat,dog,person
bottle,20.57,15.29,4.91,4.82,4.82,2.72
car,70.52,6.71,62.92,6.67,6.66,7.11
cat,72.78,6.09,6.18,60.55,6.33,5.74
dog,58.00,4.67,4.70,5.93,57.60,4.58
person,69.37,8.90,6.18,5.83,6.57,71.01


\begin{table}
\caption{explainer}
\begin{tabular}{lrrrrrr}
 & none & bottle & car & cat & dog & person \\
bottle & \background-color#081d58 \color#f1f1f1 20.57 & \background-color#1f82b9 \color#f1f1f1 15.29 & \background-color#ffffd9 \color#000000 4.91 & \background-color#ffffd9 \color#000000 4.82 & \background-color#ffffd9 \color#000000 4.82 & \background-color#ffffd9 \color#000000 2.72 \\
car & \background-color#081d58 \color#f1f1f1 70.52 & \background-color#fcfed1 \color#000000 6.71 & \background-color#22328f \color#f1f1f1 62.92 & \background-color#fcfed1 \color#000000 6.67 & \background-color#fcfed1 \color#000000 6.66 & \background-color#fafdcf \color#000000 7.11 \\
cat & \background-color#081d58 \color#f1f1f1 72.78 & \background-color#fdfed4 \color#000000 6.09 & \background-color#fdfed4 \color#000000 6.18 & \background-color#24479d \color#f1f1f1 60.55 & \background-color#fcfed3 \color#000000 6.33 & \background-color#feffd6 \color#000000 5.74 \\
dog & \background-color#081d58 \colo

,none,bottle,car,cat,dog,person
bottle,20.57,16.10,5.24,4.85,4.76,3.61
car,70.52,6.93,50.19,6.65,6.53,8.92
cat,72.78,6.17,6.39,43.74,8.99,5.85
dog,58.00,4.46,4.39,8.23,41.88,4.38
person,69.37,11.81,8.18,5.90,8.75,56.38


\begin{table}
\caption{grad_cam}
\begin{tabular}{lrrrrrr}
 & none & bottle & car & cat & dog & person \\
bottle & \background-color#081d58 \color#f1f1f1 20.57 & \background-color#216daf \color#f1f1f1 16.10 & \background-color#fdfed5 \color#000000 5.24 & \background-color#ffffd9 \color#000000 4.85 & \background-color#ffffd9 \color#000000 4.76 & \background-color#ffffd9 \color#000000 3.61 \\
car & \background-color#081d58 \color#f1f1f1 70.52 & \background-color#fbfdd0 \color#000000 6.93 & \background-color#2076b3 \color#f1f1f1 50.19 & \background-color#fcfed1 \color#000000 6.65 & \background-color#fcfed3 \color#000000 6.53 & \background-color#f7fcc6 \color#000000 8.92 \\
cat & \background-color#081d58 \color#f1f1f1 72.78 & \background-color#fdfed4 \color#000000 6.17 & \background-color#fcfed3 \color#000000 6.39 & \background-color#2ca1c2 \color#f1f1f1 43.74 & \background-color#f7fcc6 \color#000000 8.99 & \background-color#fdfed5 \color#000000 5.85 \\
dog & \background-color#081d58 \color

,none,bottle,car,cat,dog,person
bottle,20.57,18.67,10.94,12.05,10.70,9.73
car,70.52,29.75,56.66,30.27,34.43,32.42
cat,72.78,37.89,39.28,49.61,41.89,36.02
dog,58.00,19.64,20.34,23.72,45.12,22.83
person,69.37,56.06,53.39,47.33,54.81,63.52


\begin{table}
\caption{rise}
\begin{tabular}{lrrrrrr}
 & none & bottle & car & cat & dog & person \\
bottle & \background-color#081d58 \color#f1f1f1 20.57 & \background-color#243392 \color#f1f1f1 18.67 & \background-color#7cccbb \color#000000 10.94 & \background-color#59bfc0 \color#000000 12.05 & \background-color#85cfba \color#000000 10.70 & \background-color#a9ddb7 \color#000000 9.73 \\
car & \background-color#081d58 \color#f1f1f1 70.52 & \background-color#7ecdbb \color#000000 29.75 & \background-color#2351a2 \color#f1f1f1 56.66 & \background-color#7acbbc \color#000000 30.27 & \background-color#5bc0c0 \color#000000 34.43 & \background-color#69c5be \color#000000 32.42 \\
cat & \background-color#081d58 \color#f1f1f1 72.78 & \background-color#48b9c3 \color#f1f1f1 37.89 & \background-color#3fb4c4 \color#f1f1f1 39.28 & \background-color#1e83ba \color#f1f1f1 49.61 & \background-color#34a9c3 \color#f1f1f1 41.89 & \background-color#55bec1 \color#000000 36.02 \\
dog & \background-color#081d58

,none,bottle,car,cat,dog,person
bottle,20.57,18.92,11.19,13.37,13.44,17.24
car,70.52,41.25,67.02,40.69,31.87,54.31
cat,72.78,45.84,36.27,72.62,59.21,54.16
dog,58.00,41.37,34.76,57.59,61.22,58.76
person,69.37,64.71,54.41,43.42,61.55,73.59


\begin{table}
\caption{rt_saliency}
\begin{tabular}{lrrrrrr}
 & none & bottle & car & cat & dog & person \\
bottle & \background-color#081d58 \color#f1f1f1 20.57 & \background-color#21308b \color#f1f1f1 18.92 & \background-color#75c9bd \color#000000 11.19 & \background-color#36abc3 \color#f1f1f1 13.37 & \background-color#35aac3 \color#f1f1f1 13.44 & \background-color#2351a2 \color#f1f1f1 17.24 \\
car & \background-color#081d58 \color#f1f1f1 70.52 & \background-color#32a6c2 \color#f1f1f1 41.25 & \background-color#142670 \color#f1f1f1 67.02 & \background-color#34a9c3 \color#f1f1f1 40.69 & \background-color#6fc7bd \color#000000 31.87 & \background-color#225da8 \color#f1f1f1 54.31 \\
cat & \background-color#081d58 \color#f1f1f1 72.78 & \background-color#2397c1 \color#f1f1f1 45.84 & \background-color#53bdc1 \color#000000 36.27 & \background-color#081d58 \color#f1f1f1 72.62 & \background-color#234da0 \color#f1f1f1 59.21 & \background-color#2168ad \color#f1f1f1 54.16 \\
dog & \background-colo